In [1]:
#!pip install langchain_community

In [1]:
#!pip install neo4j
#!pip install python-dotenv

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

## Setup NEO4J

In [3]:
from langchain_community.graphs import Neo4jGraph
url = "bolt://localhost:7687"
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
graph = Neo4jGraph(url = url, username = username, password=password)

## Creating data set

In [4]:
import requests

import_url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(import_url).json()['query']
graph.query(
    import_query
)

[]

In [3]:
#!pip install langchain_openai

In [4]:
## Implementing vector index search


In [5]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

response = vector_index.similarity_search("How will RecommendationService be updated?")
print(response[0].page_content)


name: RecommendationFeature
description: Add feature to RecommendationService
status: In Progress


In [6]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(llm = ChatOpenAI(), chain_type = "stuff", retriever = vector_index.as_retriever())
vector_qa.invoke({"query": "How will recommendation service be updated?"})
vector_qa.invoke(
    {"query": "How many open tickets there are?"}
)
graph.query(
    "MATCH (t: Task{status: 'open'}) RETURN count(*)"
)

[{'count(*)': 0}]

In [7]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
)

cypher_chain.invoke({"query": "How many open tickets are there?"})
cypher_chain.invoke(
    {"query": "Which team has the most open tasks?"}
)
cypher_chain.invoke(
    {"query": "Which services depend on Database directly?"}
)
cypher_chain.invoke(
    {"query": "Which services depend on Database indirectly?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'open'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 0}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task)-[:ASSIGNED_TO]->(team:Team)
WHERE t.status = 'open'
RETURN team.name, COUNT(t) AS OpenTasks
ORDER BY OpenTasks DESC
LIMIT 1
Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(d:Dependency {type: 'Database'}) RETURN m.name
Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(:Dependency {name: 'Database'}) RETURN m.name
Full Context:
[{'m.name': 'PaymentService'}, {'m.name': 'UserService'}, {'m.name': 'AuthService'}, {'m.name': 'OrderService'}, {'m.name': 'CatalogService'}, {'m.name': 'InventoryService'}, {'m.name': 'PaymentService'}, {'m.name': 'OrderService'}, {'m.name': 'CatalogS

{'query': 'Which services depend on Database indirectly?',
 'result': 'PaymentService, UserService, AuthService, OrderService, CatalogService, InventoryService depend on Database indirectly.'}

In [11]:
#!pip install langchainhub

In [8]:
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub


tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions of tasks.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)
agent = create_openai_functions_agent(
    ChatOpenAI(temperature=0, model_name='gpt-4'), tools, prompt
)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "Which team is assigned to maintain PaymentService?"})
print(response)
response = agent_executor.invoke({"input": "Which tasks have optimization in their description?"})
print(response)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='